In [1]:
import xarray as xr
import stmtools

stmat = xr.open_zarr('../data/stm.zarr')
stmat

<xarray.Dataset>
Dimensions:    (points: 318225, time: 10)
Coordinates:
    azimuth    (points) int64 dask.array<chunksize=(5000,), meta=np.ndarray>
    lat        (points) float32 dask.array<chunksize=(5000,), meta=np.ndarray>
    lon        (points) float32 dask.array<chunksize=(5000,), meta=np.ndarray>
    range      (points) int64 dask.array<chunksize=(5000,), meta=np.ndarray>
  * time       (time) int64 0 1 2 3 4 5 6 7 8 9
Dimensions without coordinates: points
Data variables:
    amplitude  (points, time) float32 dask.array<chunksize=(5000, 10), meta=np.ndarray>
    complex    (points, time) complex64 dask.array<chunksize=(5000, 10), meta=np.ndarray>
    phase      (points, time) float32 dask.array<chunksize=(5000, 10), meta=np.ndarray>

In [2]:
print(stmat["lon"].shape)
print(stmat["lat"].shape)
print(stmat["amplitude"][:,0].shape)
#print(stmat["amplitude"]))[0,1:].shape)

(318225,)
(318225,)
(318225,)


In [3]:
print(stmat["lon"].values)
print(stmat["lat"].values)
print(stmat["amplitude"][:,0].values)

[4.810414 4.815807 4.818206 ... 4.962382 4.962496 4.962553]
[52.231483 52.232105 52.232384 ... 52.503086 52.5031   52.503105]
[  3300.7268  29238.707    8054.4707 ... 276654.2     64619.95
   6178.8945]


In [8]:
import laspy
import numpy as np

data_x = 100*stmat["lon"].to_numpy()
data_y = 100*stmat["lat"].to_numpy()
data_z = stmat["amplitude"][:,0].to_numpy()/2e+08
data_i = np.array(range(0, len(data_x))) % 65536

#data = np.hstack((data_x.reshape((-1, 1)), data_y.reshape((-1, 1)), data_z.reshape((-1, 1))))
#data = np.transpose(np.vstack((data_x.reshape((-1)), data_y.reshape((-1)), data_z.reshape((-1)))))
#data = np.vstack((data_x.reshape((-1)), data_y.reshape((-1)), data_z.reshape((-1))))
#dt = np.dtype({'names': ['X', 'Y', 'Z'], 'formats':['<f8', '<f8', '<f8']})
data = np.hstack((data_x.reshape((-1, 1)), data_y.reshape((-1, 1)), data_z.reshape((-1, 1)), data_i.reshape((-1, 1))))
dt = np.dtype({'names': ['X', 'Y', 'Z', 'Intensity'], 'formats':['<f8', '<f8', '<f8', '<u2']})

data

array([[4.81041382e+02, 5.22314844e+03, 1.65036345e-05, 0.00000000e+00],
       [4.81580688e+02, 5.22321045e+03, 1.46193532e-04, 1.00000000e+00],
       [4.81820587e+02, 5.22323828e+03, 4.02723526e-05, 2.00000000e+00],
       ...,
       [4.96238190e+02, 5.25030859e+03, 1.38327095e-03, 5.60780000e+04],
       [4.96249573e+02, 5.25031006e+03, 3.23099754e-04, 5.60790000e+04],
       [4.96255310e+02, 5.25031055e+03, 3.08944727e-05, 5.60800000e+04]])

In [5]:
#data_l = list(zip(data[:,0], data[:,1], data[:,2]))
data_l = list(map(tuple, data))
data_np = np.array(data_l,dt)
#data_l
data_np

array([(481.04138184, 5223.1484375 , 1.65036345e-05,     0),
       (481.58068848, 5223.21044922, 1.46193532e-04,     1),
       (481.82058716, 5223.23828125, 4.02723526e-05,     2), ...,
       (496.2381897 , 5250.30859375, 1.38327095e-03, 56078),
       (496.24957275, 5250.31005859, 3.23099754e-04, 56079),
       (496.25531006, 5250.31054688, 3.08944727e-05, 56080)],
      dtype=[('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2')])

In [6]:
print(data.shape)
print(data_np.shape)

(318225, 4)
(318225,)


In [7]:
data_np[0][0]

481.0413818359375

In [8]:
print(np.min(data, axis=0)[0:3])
print(np.max(data, axis=0)[0:3])
print(np.max(data, axis=0)[0:3]-np.min(data, axis=0)[0:3])

[ 473.31143188 5223.1484375     0.        ]
[ 503.68200684 5250.31054688    5.57022762]
[30.37057495 27.16210938  5.57022762]


In [9]:
data_write = data.copy()

# 1. Create a new header
header = laspy.LasHeader(point_format=2, version="1.2")
header.offsets = np.min(data, axis=0)[0:3]
header.scales = np.array([0.001, 0.001, 0.001])

# 2. Create a Las
las = laspy.LasData(header)

las.x = data_write[:, 0]
las.y = data_write[:, 1]
las.z = data_write[:, 2]
#las.intensity = data_i
las.intensity = data_write[:, 3]

output_filename = "/storage/MobyLe/data/test_raw.laz"
las.write(output_filename)

In [10]:
print(np.min(data, axis=0)[0:3])
print(np.max(data, axis=0)[0:3])
print(np.max(data, axis=0)[0:3]-np.min(data, axis=0)[0:3])

[ 473.31143188 5223.1484375     0.        ]
[ 503.68200684 5250.31054688    5.57022762]
[30.37057495 27.16210938  5.57022762]


In [3]:
import numpy as np

di = np.dtype(np.int32)

#data_pixel_x = stmat["azimuth"].to_numpy(dtype=di)
#data_pixel_y = stmat["range"].to_numpy(dtype=di)
data_pixel_x = stmat["azimuth"].to_numpy()
data_pixel_y = stmat["range"].to_numpy()
data_pixel_z = stmat["amplitude"][:,0].to_numpy()/2e+08
data_pixel_i = np.array(range(0, len(data_pixel_x))) % 65536

data_pixel = np.hstack((data_pixel_x.reshape((-1, 1)), data_pixel_y.reshape((-1, 1)), data_pixel_z.reshape((-1, 1)), data_pixel_i.reshape((-1, 1))))
#data_pixel = np.hstack((data_pixel_x.reshape((-1, 1)), data_pixel_y.reshape((-1, 1))))
#dpt = np.dtype({'names': ['X', 'Y', 'Z', 'Intensity'], 'formats':['<f8', '<f8', '<f8', '<u2']})

data_pixel

array([[0.00000000e+00, 3.10000000e+01, 1.65036345e-05, 0.00000000e+00],
       [0.00000000e+00, 1.24000000e+02, 1.46193532e-04, 1.00000000e+00],
       [0.00000000e+00, 1.65000000e+02, 4.02723526e-05, 2.00000000e+00],
       ...,
       [1.99900000e+03, 3.99200000e+03, 1.38327095e-03, 5.60780000e+04],
       [1.99900000e+03, 3.99400000e+03, 3.23099754e-04, 5.60790000e+04],
       [1.99900000e+03, 3.99500000e+03, 3.08944727e-05, 5.60800000e+04]])

In [6]:
# This is the actually important (functional) part. The rest is mostly for visualizing the results.


import pymorton as pm

data_pixel[:, 3] = [pm.interleave(int(p[1]), int(p[0])) % 65536 for p in data_pixel]
#data_pixel[:, 3] = [pm.interleave(int(p[1]), int(p[0])) for p in data_pixel] # Computing the Morton code.
data_pixel[:, 3]

#mortoncode = pm.interleave(100, 200)
#mortoncode

array([  341.,  5456., 17425., ..., 57834., 57838., 57839.])

In [9]:
import laspy

# 1. Create a new header
header = laspy.LasHeader(point_format=2, version="1.2")
header.offsets = np.min(data, axis=0)[0:3]
header.scales = np.array([0.001, 0.001, 0.001])

# 2. Create a Las
las = laspy.LasData(header)

las.x = data[:, 0]
las.y = data[:, 1]
las.z = data[:, 2]
#las.intensity = data_i
las.intensity = data_pixel[:, 3]

output_filename = "/storage/MobyLe/data/test_pymorton.laz"
las.write(output_filename)

In [13]:
# This is the actually important (functional) part. The rest is mostly for visualizing the results.


import pdal

#arr = np.array(data,dt)
arr = data_np

pipeline_morton = pdal.Filter.mortonorder().pipeline(arr)
print(pipeline_morton.execute())

arr

ModuleNotFoundError: No module named 'pdal'

In [224]:
print(arr)
print('---------------')
print(arr[0])

[(481.04138184, 5223.1484375 , 1.65036345e-05,     0)
 (481.58068848, 5223.21044922, 1.46193532e-04,     1)
 (481.82058716, 5223.23828125, 4.02723526e-05,     2) ...
 (496.2381897 , 5250.30859375, 1.38327095e-03, 56078)
 (496.24957275, 5250.31005859, 3.23099754e-04, 56079)
 (496.25531006, 5250.31054688, 3.08944727e-05, 56080)]
---------------
(481.04138184, 5223.1484375, 1.65036345e-05, 0)


In [155]:
print(np.min(arr[0], axis=0)[0:3])
print(np.max(arr[0], axis=0)[0:3])
print(np.max(arr[0], axis=0)[0:3]-np.min(arr, axis=0)[0:3])

UFuncTypeError: ufunc 'minimum' did not contain a loop with signature matching types (dtype([('X', '<f8'), ('Y', '<f8'), ('Z', '<f8')]), dtype([('X', '<f8'), ('Y', '<f8'), ('Z', '<f8')])) -> None

In [225]:
arr_morton = pipeline_morton.arrays[0]
arr_morton["Intensity"] = np.array(range(0, len(arr_morton))) % 65536
arr_morton["Intensity"]

arr_morton

array([(480.88537598, 5223.18115234, 9.95832033e-06,     0),
       (480.86734009, 5223.24267578, 5.43377595e-04,     1),
       (480.87316895, 5223.24316406, 7.62824842e-04,     2), ...,
       (496.33850098, 5250.04052734, 1.16414425e-03, 56078),
       (496.34228516, 5250.02832031, 6.89363107e-04, 56079),
       (496.33468628, 5250.05273438, 1.64161855e-03, 56080)],
      dtype=[('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2')])

In [124]:
print(np.min(arr_morton.array, axis=0)[0:3])
print(np.max(arr_morton, axis=0)[0:3])

AttributeError: 'numpy.ndarray' object has no attribute 'array'

In [226]:
output_filename = "/storage/MobyLe/data/test_morton.laz"
pipeline = pdal.Writer.las(
    filename=output_filename,
    offset_x="auto",
    offset_y="auto",
    offset_z="auto",
    scale_x=0.001,
    scale_y=0.001,
    scale_z=0.001,
).pipeline(arr_morton)
print(pipeline.execute())

318225


In [229]:
import laspy

las = laspy.read(output_filename)

print(las.header)
print(las.header.offsets)
print(las.header.scales)

print(las.x)


# 1. Create a new header
#header = laspy.LasHeader(point_format=2, version="1.2")
#header.offsets = np.min(data, axis=0)[0:3]
#header.scales = np.array([0.001, 0.001, 0.001])

# 2. Create a Las
#las = laspy.LasData(header)

#las.x = data_write[:, 0]
#las.y = data_write[:, 1]
#las.z = data_write[:, 2]
#las.intensity = data_write[:, 3]

<LasHeader(1.2, <PointFormat(3, 0 bytes of extra dims)>)>
[4.88496719e+02 5.23672949e+03 2.78511381e+00]
[0.001 0.001 0.001]
<ScaledArrayView([480.88571936 480.86771936 480.87271936 ... 496.33871936 496.34271936
 496.33471936])>


In [2]:
subset = stmat.stm.subset(method='threshold', var='amplitude', threshold='>1')
subset

<xarray.Dataset>
Dimensions:    (points: 316762, time: 10)
Coordinates:
    azimuth    (points) int64 dask.array<chunksize=(5000,), meta=np.ndarray>
    lat        (points) float32 dask.array<chunksize=(5000,), meta=np.ndarray>
    lon        (points) float32 dask.array<chunksize=(5000,), meta=np.ndarray>
    range      (points) int64 dask.array<chunksize=(5000,), meta=np.ndarray>
  * time       (time) int64 0 1 2 3 4 5 6 7 8 9
Dimensions without coordinates: points
Data variables:
    amplitude  (points, time) float32 dask.array<chunksize=(5000, 10), meta=np.ndarray>
    complex    (points, time) complex64 dask.array<chunksize=(5000, 10), meta=np.ndarray>
    phase      (points, time) float32 dask.array<chunksize=(5000, 10), meta=np.ndarray>

In [14]:
#pipeline = pdal.Filter.mortonorder().pipeline(subset)
#pipeline = pdal.Filter.sort(dimension="lat").pipeline(subset.to_array())
#print(pipeline.execute())

In [1]:
import pdal

data = "/storage/nD_PointClouds/data/entwine/red-rocks-mpi.laz"

pipeline_read = pdal.Reader.las(filename=data).pipeline()
print(pipeline_read.execute())

ModuleNotFoundError: No module named 'pdal'

In [198]:
# Get the data from the first array
# [array([(637012.24, 849028.31, 431.66, 143, 1,
# 1, 1, 0, 1,  -9., 132, 7326, 245380.78254963,  68,  77,  88),
# dtype=[('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2'),
# ('ReturnNumber', 'u1'), ('NumberOfReturns', 'u1'), ('ScanDirectionFlag', 'u1'),
# ('EdgeOfFlightLine', 'u1'), ('Classification', 'u1'), ('ScanAngleRank', '<f4'),
# ('UserData', 'u1'), ('PointSourceId', '<u2'),
# ('GpsTime', '<f8'), ('Red', '<u2'), ('Green', '<u2'), ('Blue', '<u2')])
arr = pipeline_read.arrays[0]
arr

array([(482161.69, 4390463.5, 1954.21, 0, 1, 1, 0, 0, 0, 0., 0, 0, 0., 0, 0,  41,  40,  37),
       (482162.  , 4390463.5, 1954.04, 0, 1, 1, 0, 0, 0, 0., 0, 0, 0., 0, 0,  26,  26,  24),
       (482162.34, 4390463.5, 1953.68, 0, 1, 1, 0, 0, 0, 0., 0, 0, 0., 0, 0,  20,  21,  20),
       ...,
       (482720.16, 4390812.5, 1858.78, 0, 1, 1, 0, 0, 0, 0., 0, 0, 0., 0, 0,  81,  72,  65),
       (482586.69, 4390873.5, 1918.74, 0, 1, 1, 0, 0, 0, 0., 0, 0, 0., 0, 0, 176, 143, 114),
       (482588.38, 4391016. , 1894.7 , 0, 1, 1, 0, 0, 0, 0., 0, 0, 0., 0, 0, 125, 118,  98)],
      dtype=[('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2'), ('ReturnNumber', 'u1'), ('NumberOfReturns', 'u1'), ('ScanDirectionFlag', 'u1'), ('EdgeOfFlightLine', 'u1'), ('Classification', 'u1'), ('ScanAngleRank', '<f4'), ('UserData', 'u1'), ('PointSourceId', '<u2'), ('GpsTime', '<f8'), ('ScanChannel', 'u1'), ('ClassFlags', 'u1'), ('Red', '<u2'), ('Green', '<u2'), ('Blue', '<u2')])

In [199]:
# Back to using the originally read point cloud "arr".

pipeline_morton = pdal.Filter.mortonorder().pipeline(arr)
print(pipeline_morton.execute())

4004326


In [200]:
import numpy as np

arr_morton = pipeline_morton.arrays[0]
arr_morton["Intensity"] = np.array(range(0, len(arr_morton))) % 65536
arr_morton["Intensity"]

array([   0,    1,    2, ..., 6627, 6628, 6629], dtype=uint16)

In [201]:
output_data = "/storage/nD_PointClouds/data/entwine/red-rocks-mpi_morton.laz"
pipeline = pdal.Writer.las(
    filename=output_data,
    offset_x="auto",
    offset_y="auto",
    offset_z="auto",
    scale_x=0.01,
    scale_y=0.01,
    scale_z=0.01,
).pipeline(arr_morton)
print(pipeline.execute())

4004326


In [3]:
# Filter out entries that have Z > 1940
z = arr[arr["Z"] > 1960]
print(len(z))

674215


In [4]:
# Now use pdal to clamp points that have z 1970 <= v < 1980
pipeline_clamp = pdal.Filter.range(limits="Z[1970:1980)").pipeline(z)
print(pipeline_clamp.execute())  # 387 points
clamped = pipeline_clamp.arrays[0]

153927


In [5]:
pipeline_sort = pdal.Filter.sort(dimension="X").pipeline(arr)
print(pipeline_sort.execute())

4004326
